# Data Preprocessing Example

In this example, we show how to preprocess the data and convert ordinal or norminal data to numerical data. We use the following data set:

https://archive.ics.uci.edu/ml/datasets/Adult

The data was extracted by Barry Becker from the 1994 Census database. Prediction task is to determine whether a person makes over 50K a year. 

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

## Loading Data

Note that you can use **na_values** parameter to convert a specific string (such as ?, #NA# etc) to NA value. You can also use regular expression delimiter to "eat" extra spaces in the data file. 

In [2]:
df = pd.read_csv('https://archive.ics.uci.edu'
                 '/ml/machine-learning-databases/adult'
                 '/adult.data', header=None, 
                 delimiter=r",\s*" ,na_values='?', engine='python')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       32561 non-null  int64 
 1   1       30725 non-null  object
 2   2       32561 non-null  int64 
 3   3       32561 non-null  object
 4   4       32561 non-null  int64 
 5   5       32561 non-null  object
 6   6       30718 non-null  object
 7   7       32561 non-null  object
 8   8       32561 non-null  object
 9   9       32561 non-null  object
 10  10      32561 non-null  int64 
 11  11      32561 non-null  int64 
 12  12      32561 non-null  int64 
 13  13      31978 non-null  object
 14  14      32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [3]:
df.isna().sum()

0        0
1     1836
2        0
3        0
4        0
5        0
6     1843
7        0
8        0
9        0
10       0
11       0
12       0
13     583
14       0
dtype: int64

From the file *adult.names*, we can get the information about each columns:

>50K, <=50K.

age: continuous.
workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.

fnlwgt: continuous.

education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 
7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.

education-num: continuous.

marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.

occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.

relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.

race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.

sex: Female, Male.

capital-gain: continuous.

capital-loss: continuous.

hours-per-week: continuous.

native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

## Add column names. 

This step is not required. But it is helpful for understanding what each column is. 

In [4]:
df.columns=['age', 'workclass', 'fnlwgt', 'education', 'education_num','marital-status',
           'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
           'hours_per_week', 'native_country', 'target']

In [5]:
df.head()

,age,workclass,fnlwgt,education,education_num,marital-status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## Check how much data is missing

This step is to check how much data is missing and how to handle missing data. Generally, missing data can be handled by three different strategies:

1. Remove samples (rows) with missing data.
2. Remove features (columns) that miss too much data.
3. Impute data. 

The strategies to be used depend on how much data is missing, how important the feature is or whether imputing data is possible. 

In [6]:
print(df.isna().sum())
print(df.shape)

age                  0
workclass         1836
fnlwgt               0
education            0
education_num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital_gain         0
capital_loss         0
hours_per_week       0
native_country     583
target               0
dtype: int64
(32561, 15)


Now we can see that we miss about 5% data data for workclass, occupation and native_country. So method 1 (removing rows) probably a better choice. We can do this by the following code:

In [7]:
df1 = df.dropna(axis=0)
df1.shape

(30162, 15)

If you prefer the second method, we can use the following code to do so:

In [8]:
df2 = df.dropna(axis=1, thresh=df.shape[0]-1800) #drop features that miss more than 1800 samples.
df2 = df2.dropna(axis=0)  #drop samples that still has missing data
df2.shape

(31978, 13)

Now we analyze each column:

1. Age: Numerical.
* Workclass: Nominal
* fnlwgt (Final Weight): Numerical.  
* Education: Ordinal
* Education-num: Numerical
* Marital-status: Nominal
* Occupation: Nominal
* Relationship: Nominal
* Race: Nominal
* Sex: Binary ordinal
* Captial-gain: Numerical
* Capital-loss: Numerical.
* Hours-per-week: Numerical
* Native-country: Nominal

18. Nominal Target

All these features will be used.

## Process ordinal data:

In [9]:
df1.head()

,age,workclass,fnlwgt,education,education_num,marital-status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [10]:
# Check all possible values for education.
np.unique(df1['education'])

array(['10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th',
       'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad',
       'Masters', 'Preschool', 'Prof-school', 'Some-college'],
      dtype=object)

In [11]:
#disable copyonwrite warning. Optional.
pd.options.mode.chained_assignment = None  # default='warn'

In [12]:
#map the values:
edu_mapping = {'Preschool':0, '1st-4th':1, '5th-6th':2,'7th-8th':3, '9th':4, 
              '10th':5, '11th':6, '12th':7, 'HS-grad':8, 'Some-college':9,
              'Assoc-voc':10,'Assoc-acdm':11,  'Bachelors':12, 'Masters':13,
               'Doctorate':14, 'Prof-school':15}
df1['education']=df1['education'].map(edu_mapping)

In [13]:
df1.head()

,age,workclass,fnlwgt,education,education_num,marital-status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,target
0,39,State-gov,77516,12,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,12,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,8,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,6,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,12,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [14]:
#Convert binary categorical feature sex
sex_mapping = {label: idx for idx, label in enumerate(np.unique(df['sex']))}
df1['sex'] = df1['sex'].map(sex_mapping)
sex_mapping

{'Female': 0, 'Male': 1}

In [15]:
#Convert binary categorical target
target_mapping = {label: idx for idx, label in enumerate(np.unique(df['target']))}
df1['target'] = df1['target'].map(target_mapping)
target_mapping

{'<=50K': 0, '>50K': 1}

In [16]:
df1.head()

,age,workclass,fnlwgt,education,education_num,marital-status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,target
0,39,State-gov,77516,12,13,Never-married,Adm-clerical,Not-in-family,White,1,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,12,13,Married-civ-spouse,Exec-managerial,Husband,White,1,0,0,13,United-States,0
2,38,Private,215646,8,9,Divorced,Handlers-cleaners,Not-in-family,White,1,0,0,40,United-States,0
3,53,Private,234721,6,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,1,0,0,40,United-States,0
4,28,Private,338409,12,13,Married-civ-spouse,Prof-specialty,Wife,Black,0,0,0,40,Cuba,0


## Encode nominal columns

In [17]:
# Label encoding with Pandas get_dummies method
df3 = pd.get_dummies(df1.iloc[:, :-1], drop_first=True)
df3.head()

,age,fnlwgt,education,education_num,sex,capital_gain,capital_loss,hours_per_week,workclass_Local-gov,workclass_Private,...,native_country_Portugal,native_country_Puerto-Rico,native_country_Scotland,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia
0,39,77516,12,13,1,2174,0,40,False,False,...,False,False,False,False,False,False,False,True,False,False
1,50,83311,12,13,1,0,0,13,False,False,...,False,False,False,False,False,False,False,True,False,False
2,38,215646,8,9,1,0,0,40,False,True,...,False,False,False,False,False,False,False,True,False,False
3,53,234721,6,7,1,0,0,40,False,True,...,False,False,False,False,False,False,False,True,False,False
4,28,338409,12,13,0,0,0,40,False,True,...,False,False,False,False,False,False,False,False,False,False


In [18]:
df3.shape

(30162, 82)

## Get X and y as Numpy arrays.

In [19]:
X = df3.values
y = df1.iloc[:, -1].values

In [20]:
print(X[:500:100, :10])

[[39 77516 12 13 1 2174 0 40 False False]
 [42 124692 8 9 1 0 0 40 False True]
 [37 635913 12 13 1 0 0 60 False True]
 [29 200381 6 7 0 0 0 40 False True]
 [40 204116 12 13 0 2174 0 40 False False]]


## Split data into training and test data sets

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, 
                     test_size=0.25,
                     stratify=y,
                     random_state=1)

In [22]:
print(X_train.shape)
print(X_test.shape)

(22621, 82)
(7541, 82)


### Check how many samples are in each class

In [23]:
print(np.unique(y)) 

[0 1]


In [24]:
print(np.sum(y==0))
print(np.sum(y==1))

22654
7508


If there are many classes, you can also use the following code to get all counts.

In [25]:
{i:np.sum(y==i) for i in np.unique(y)}

{0: 22654, 1: 7508}

## Scikit Learn

We use Scikit Learn toolbox to create and train the model

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

### Normalize the data

We usually need to normalize data before train the model. The nomalization parameters (mean and standard deviation) can only be calculated from training data. It will also be applied to the test data while we test models.

In [27]:
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
#scaler.fit(X_train)
#X_train_std=scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

In [28]:
print(X_train[:5,:10])
print(X_train_std[:5,:10])

[[37 200374 12 13 1 0 1902 40 False True]
 [59 131916 8 9 0 0 0 40 False True]
 [36 113337 9 10 1 0 0 42 True False]
 [22 110946 9 10 0 0 0 40 False True]
 [56 274111 12 13 1 0 0 45 False True]]
[[-0.10789786  0.10140411  1.12006458  1.12783734  0.6901286  -0.14872906
   4.47495381 -0.07905133 -0.27150265  0.59299004]
 [ 1.56182653 -0.54641825 -0.43803159 -0.4378459  -1.44900529 -0.14872906
  -0.21797694 -0.07905133 -0.27150265  0.59299004]
 [-0.18379443 -0.72223249 -0.04850755 -0.04642509  0.6901286  -0.14872906
  -0.21797694  0.08784588  3.6832053  -1.68636896]
 [-1.24634631 -0.74485867 -0.04850755 -0.04642509 -1.44900529 -0.14872906
  -0.21797694 -0.07905133 -0.27150265  0.59299004]
 [ 1.33413684  0.79918198  1.12006458  1.12783734  0.6901286  -0.14872906
  -0.21797694  0.33819168 -0.27150265  0.59299004]]


Now we create a logistic regression model. You can see what parameters can be used by checking the help page.

In [29]:
#?LogisticRegression

In [30]:
logistic_model = LogisticRegression(solver='lbfgs', 
                                    multi_class='auto', tol=1e-4, max_iter = 4000, C=1)

The most important parameter here is C, which is the inverse of the regulation. Smaller values specify stronger regularization. Therefore causes less overfitting but potentially more underfitting.

In [31]:
logistic_model.fit(X_train_std, y_train)
print("Training set score: %f" % logistic_model.score(X_train_std, y_train))
print("Testing set score: %f" % logistic_model.score(X_test_std, y_test))

Training set score: 0.848813
Testing set score: 0.850418


c:\Users\leqian\anaconda3\envs\py312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [32]:
y_test_pred = logistic_model.predict(X_test_std)

In [33]:
y_test_diff = y_test[y_test_pred!=y_test]
print(y_test_diff)

[0 1 0 ... 1 0 1]
